In [15]:
#limpa df, comeca etc

import pandas as pd
import numpy as np
import re

df = pd.read_csv('data/IMDB.csv' , encoding='utf-8')

def clean_t(t):
    t_clean = re.sub(r'<.*?>', '', t)
    t_clean = re.sub(r'[^\w\s]', '', t_clean)
    return t_clean

df['review'] = df['review'].apply(clean_t)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [56]:
#fazendo o treino
df_treino = df.iloc[:10]
df_treino.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [59]:
#fazendo a dict de palavras nao repetidas


# Inicializa os dicionários de contagem para positivos e negativos
contagem_palavras = {}

# Itera sobre os textos e sentimentos
for texto, sentimento in zip(df_treino['review'], df_treino['sentiment']):
    palavras = texto.lower().split()  # Divide o texto em palavras
    for palavra in palavras:
        if palavra not in contagem_palavras:
            contagem_palavras[palavra] = {'positive': 0, 'negative': 0}
        contagem_palavras[palavra][sentimento] += 1

print(contagem_palavras)


{'one': {'positive': 10, 'negative': 3}, 'of': {'positive': 28, 'negative': 10}, 'the': {'positive': 65, 'negative': 28}, 'other': {'positive': 2, 'negative': 0}, 'reviewers': {'positive': 1, 'negative': 0}, 'has': {'positive': 4, 'negative': 1}, 'mentioned': {'positive': 1, 'negative': 0}, 'that': {'positive': 7, 'negative': 6}, 'after': {'positive': 1, 'negative': 1}, 'watching': {'positive': 4, 'negative': 1}, 'just': {'positive': 3, 'negative': 3}, '1': {'positive': 1, 'negative': 0}, 'oz': {'positive': 5, 'negative': 0}, 'episode': {'positive': 2, 'negative': 0}, 'youll': {'positive': 1, 'negative': 0}, 'be': {'positive': 9, 'negative': 2}, 'hooked': {'positive': 1, 'negative': 0}, 'they': {'positive': 4, 'negative': 0}, 'are': {'positive': 8, 'negative': 3}, 'right': {'positive': 3, 'negative': 0}, 'as': {'positive': 6, 'negative': 4}, 'this': {'positive': 14, 'negative': 9}, 'is': {'positive': 26, 'negative': 11}, 'exactly': {'positive': 2, 'negative': 0}, 'what': {'positive': 4

In [14]:
df_teste = df.iloc[25000:]
df_teste.head()

,review,sentiment
25000,This movie was bad from the start The only pur...,negative
25001,God I never felt so insulted in my whole life ...,negative
25002,Not being a fan of the Coen Brothers or George...,positive
25003,The movie Andaz Apna Apna in my books is the t...,positive
25004,I have to say I was really looking forward on ...,negative
